# Install Dependencies

In [1]:
pip install trl transformers accelerate peft install datasets bitsandbytes einops

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
import pandas as pd

In [4]:
from datasets import Dataset

dataset_dir = '/kaggle/input/poemgeneratordataset/PoemGeneratorDataset.csv'

In [5]:
df = pd.read_csv(dataset_dir)
df = pd.DataFrame(df)
df.head()

,Concept,Poem,text
0,The Sunday Afternoon,"Sunday's sun, a warm caress,\r\nFamilies gathe...",###Human:\r\ngenerate a poem for the topic The...
1,Whispering Willow,"Willow's branches, a gentle sigh,\r\nBeneath i...",###Human:\r\ngenerate a poem for the topic Whi...
2,Eternal Love,"Love's flame, forever bright,\r\nGuiding us th...",###Human:\r\ngenerate a poem for the topic Ete...
3,Starry Night,"Starry night, a cosmic dance,\r\nHeavenly wond...",###Human:\r\ngenerate a poem for the topic Sta...
4,Autumn's Embrace,"Autumn leaves in colors arrayed,\r\nMemories c...",###Human:\r\ngenerate a poem for the topic Aut...


In [6]:
dataset = Dataset.from_pandas(df, split='train')

In [7]:
dataset

Dataset({
    features: ['Concept', 'Poem', 'text'],
    num_rows: 68
})

## Loading the model

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [9]:
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

## Load the Tokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
# Parameter-Efficient Fine-Tuning (PEFT)
from peft import LoraConfig, get_peft_model

# LoRA - Low Rank Adaptation of LLM

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer

In [14]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

# Ste training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to=None
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
from trl import SFTTrainer      # Supervised Fine Tuning
   
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [16]:
# Upcasting the layer norms to have for stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# Train the model

In [17]:
# trainer.train()

In [18]:
# Save the model
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [19]:
# Load the trained LLM

lora_config = LoraConfig.from_pretrained('/kaggle/input/poemgeneratormodel/outputs')
newmodel = get_peft_model(model, lora_config)

In [25]:
text = "Eternal love"
device ="cuda:0"

In [26]:
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = newmodel.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Eternal love is a beautiful thing. nobody can deny that.
I'm not sure if I'm in love with you or if I'm in love with the idea of being in love with you.
I'm in love with you and all your little quirks.
I'm in love with you and I'm not afraid to admit it.
I'm in love with you and I'm not afraid to admit it. I'm in love with you
